# PyRAF: Fully loaded

This notebook illustrates how to find all of the tasks available in the current IRAF/PyRAF distribution.  

In [1]:
from __future__ import print_function
from pyraf import iraf
from stsci.tools.irafglobals import IrafTask, IrafPkg, IrafError
from types import FunctionType
from astropy.table import Table
import json
import xlsxwriter

In [2]:
def load_all_packages():
    """Recursively load all IRAF packages into the current namespace
    
    Parameters
    ----------
    None
    
    Returns
    -------
    errored_packages : dict
        package name and exception for any packages that could not be loaded
    
    """
    
    errored_packages = {}
    
    previous_size = 0
    
    #-- Iterate until no new packages have been loaded
    while len(iraf.mmdict) > previous_size:
        previous_size = len(iraf.mmdict)
       
        for name in list(iraf.mmdict.keys()):
            obj = iraf.mmdict[name]
            
            if isinstance(obj, IrafPkg):
                try:
                    print("Loading {}".format(name))
                    obj()
                except (AttributeError, ImportError, IrafError, EOFError) as e:
                    errored_packages[get_full_path(name)] = e
                    print("ERROR WITH: {}".format(name))
                    print("   BECAUSE: {}".format(e.message))
                    
    return errored_packages

In [3]:
def get_full_path(taskname):
    """Assemble full path name for a given IRAF task
    
    Find the task's parent, then each parent's parent until the top-level 
    ("clpackage") is reached.  All parents and the task will be concatenated and
    separated by ".". 
    
    Parameters
    ----------
    taskname : str
        Name of any IRAF task loaded into the namespace
        
    Returns
    -------
    outname : str
        Full path to the input task through the namespace
        
    Examples
    --------
        >>> get_full_path('mscombine')
        'clpackage.stsdas.toolbox.imgtools.mstools.mscombine'
    """
    
    outname = getattr(iraf, taskname).getPkgname() + '.' + taskname
    
    while outname.split('.')[0] != 'clpackage':
        outname = getattr(iraf, outname.split('.')[0]).getPkgname() + '.' + outname
        
    return outname

In [4]:
def find_all_tasks():
    """Find all tasks loaded into the current namespace
    
    Ignoring built-in functions, assemble the sorted list
    of all currently available IRAF tasks.
    
    Parameters
    ----------
    None
    
    Returns
    -------
    all_tasks : list
        Full path of all currently available tasks.
    """
    
    all_tasks = []
    for name in list(iraf.mmdict.keys()):
        obj = iraf.mmdict[name]
            
        #-- cannot check only if instance of IrafTask, as an IrafTask
        #-- also are subclasses of IrafPkg.
        if isinstance(obj, IrafTask):
            print(get_full_path(name))
            complete_task = get_full_path(name)
            
            #-- weed out things we don't care about
            
            #-- cmd line functions like cp, wc, vi, etc
            if complete_task.startswith('clpackage.user.'):
                continue
                
            #-- Iraf or Pyraf internal utilites
            if complete_task.startswith('clpackage.system'):
                continue
                
            #-- logout function
            if complete_task == 'clpackage._logout':
                continue
            
            #-- cl function
            if complete_task == 'clpackage.cl':
                continue
                
            #-- stuff to build packages it seems
            if complete_task.startswith('clpackage.softools.'):
                continue
            
            all_tasks.append(get_full_path(name))
            
    return sorted(all_tasks)

## Loading every package and sub-package we can find

A single call to `load_all_packages()` will continuously load every package and sub-package it can find untill no more sub-packages have been added to the namespace.  It will also return any failed packages, so that we can diagnose any failures.

In [5]:
failed = load_all_packages()

Loading nfextern
nfextern/:
 ace/           msctools/       newfirm/        odi/
Loading tv
Loading imfit
Loading song
ERROR WITH: song
   BECAUSE: Undefined IRAF task `rvx'
Loading obsolete
obsolete/:
 imtitle        ofixpix         oimstatistics   owfits
 mkhistogram    oimcombine      orfits          radplt
Loading guiapps
guiapps/:
 demo           spt/            xapphot/        xhelp           xrv/
Loading cfh12k
This is the initial release of the IRAF FITSUTIL package
to include support for FITS tile compression via 'fpack'.
Please send comments and questions to seaman@noao.edu.

cfh12k/:
 hdrcfh12k      setcfh12k
Loading nttools
Loading iis
iis/:
 blink          erase           monochrome      window
 cv             frame           pseudocolor     zoom
 cvl            lumatch         rgb
Loading plot
Loading rvsao

#-----------------------------------------------------------+
#           RVSAO Radial Velocity Analysis Package          |
#           Smithsonian Astrophysical Obse

/Users/ely/miniconda3/envs/iraf/lib/python2.7/site-packages/ipykernel/__main__.py:33: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6



 aidpars@       dopcor          refspectra      scopy           slist
 autoidentify   fitprofs        reidentify      sensfunc        specplot
 bplot          identify        rspectext       setairmass      specshift
 calibrate      lcalib          sapertures      setjd           splot
 continuum      mkspec          sarith          sfit            standard
 deredden       names           sbands          sflip           telluric
 dispcor        ndprep          scombine        sinterp         wspectext
 disptrans      odcombine       scoords         skytweak
Loading immatch
Loading cirred
cirred/:
 atmo_cor       do_osiris       fixfits         osiris
 calc_off       do_wcs          irdiff          shift_comb
 clearim        extra           maskbad         sky_sub
 do_ccmap       fixbad          med             spec_comb
Loading cutoutpkg
Parent package for cutout tasks, added in Ureka mainly to provide IRAF help

cutoutpkg/:
 cutout         ndwfsget
Loading proto
Loading gmisc
gmisc/:

In [6]:
print("Found {} failed packages.".format(len(failed)))
print("-"*20)
for item in failed:
    print(item, ' ->', failed[item])

Found 11 failed packages.
--------------------
clpackage.song  -> Undefined IRAF task `rvx'
clpackage.stsdas.analysis.dither  -> No module named pydrizzle
clpackage.stsdas.hst_calib.nicmos  -> No module named nictools
clpackage.mscdb  -> Cannot find executable for task mscdb
Tried /Users/ely/miniconda3/envs/iraf/iraf/bin.macosx/mscdb.cl, /Users/ely/miniconda3/envs/iraf/variants/common//iraf/mscdb/mscdb.cl
clpackage.noao.twodspec.apextract.apdemos  -> EOF on parameter prompt
clpackage.noao.obsutil.kpno  -> Undefined variable `spectimedb' in string `spectimedb$'
clpackage.stsdas.hst_calib.wfpc  -> No module named wfpc2tools
clpackage.stsdas.sobsolete  -> Cannot find executable for task sobsolete
Tried /Users/ely/miniconda3/envs/iraf/variants/common//iraf/stsci_iraf//stsdas/bin/sobsolete.cl, /Users/ely/miniconda3/envs/iraf/variants/common//iraf/stsci_iraf//stsdas/pkg/sobsolete/sobsolete.cl
clpackage.kepler  -> No module named pyfits
clpackage.xray.xlocal  -> Cannot find executable for tas

## After all packages have been loaded, now we need to identify every task

In [7]:
every_loaded_task = find_all_tasks()
print("Found {} loaded tasks.".format(len(every_loaded_task)))

clpackage.noao.digiphot.photcal.mkapfile
clpackage.noao.imred.echelle.doarcs
clpackage.upsqiid.sqparse
clpackage.cirred.sky_sub
clpackage.stsdas.hst_calib.synphot.obsmode
clpackage.stsdas.hst_calib.paperprod.opp_expsum
clpackage.noao.digiphot.ptools.psort
clpackage.images.tv.jimexam
clpackage.plot.pcols
clpackage.gemini.gnirs.nswhelper
clpackage.noao.digiphot.photcal.imgroup
clpackage.plot.calcomp
clpackage.gemini.gnirs.nsfitcoords
clpackage.upsqiid.stdproc
clpackage.nfextern.xtalk.xtalkcor
clpackage.stsdas.hst_calib.paperprod.opp_acq
clpackage.stsdas.analysis.isophote.isoplot
clpackage.plot.phistogram
clpackage.nfextern.newfirm.nfproc
clpackage.finder.finderlog
clpackage.cirred.do_ccmap
clpackage.lists.rimcursor
clpackage.xray.xspatial.eintools.cat_make
clpackage.gemini.gnirs.nssky
clpackage.guiapps.xapphot.cenpars
clpackage.noao.nproto.slitpic
clpackage.rvsao.rvrelearn
clpackage.proto.color.rgbdisplay
clpackage.stsdas.analysis.restore.adaptive
clpackage.stsdas.contrib.spfitpkg.dbcrea

# Dumping to JSON

In [8]:
base_tree = {'name': 'clpackage', 
             'children': []}

for task in every_loaded_task:
    tree = base_tree
    
    print('\n#---- {} ----#'.format(task))
    
    for item in task.split('.')[1:]:
        print("Searching for {}".format(item))
        found = False
        
        for row in tree['children']:
            if row['name'] == item:
                print("Found {} as child to {}, stepping down".format(item, tree['name']))
                tree = row
                found = True
        
        if not found:
            print("Adding {} as child to {}".format(item, tree['name']))
            new = {'name': item, 
                   'children': []}
            tree['children'].append(new)
            tree = new


#---- clpackage.adccdrom ----#
Searching for adccdrom
Adding adccdrom as child to clpackage

#---- clpackage.adccdrom.catalog ----#
Searching for adccdrom
Found adccdrom as child to clpackage, stepping down
Searching for catalog
Adding catalog as child to adccdrom

#---- clpackage.adccdrom.spectra ----#
Searching for adccdrom
Found adccdrom as child to clpackage, stepping down
Searching for spectra
Adding spectra as child to adccdrom

#---- clpackage.adccdrom.tbldb ----#
Searching for adccdrom
Found adccdrom as child to clpackage, stepping down
Searching for tbldb
Adding tbldb as child to adccdrom

#---- clpackage.cfh12k ----#
Searching for cfh12k
Adding cfh12k as child to clpackage

#---- clpackage.cfh12k.hdrcfh12k ----#
Searching for cfh12k
Found cfh12k as child to clpackage, stepping down
Searching for hdrcfh12k
Adding hdrcfh12k as child to cfh12k

#---- clpackage.cfh12k.setcfh12k ----#
Searching for cfh12k
Found cfh12k as child to clpackage, stepping down
Searching for setcfh12k
A

In [9]:
base_tree

{'children': [{'children': [{'children': [], 'name': 'catalog'},
    {'children': [], 'name': 'spectra'},
    {'children': [], 'name': 'tbldb'}],
   'name': 'adccdrom'},
  {'children': [{'children': [], 'name': 'hdrcfh12k'},
    {'children': [], 'name': 'setcfh12k'}],
   'name': 'cfh12k'},
  {'children': [{'children': [], 'name': 'atmo_cor'},
    {'children': [], 'name': 'calc_off'},
    {'children': [], 'name': 'clearim'},
    {'children': [], 'name': 'do_ccmap'},
    {'children': [], 'name': 'do_osiris'},
    {'children': [], 'name': 'do_wcs'},
    {'children': [], 'name': 'extra'},
    {'children': [], 'name': 'fixbad'},
    {'children': [], 'name': 'fixfits'},
    {'children': [], 'name': 'irdiff'},
    {'children': [], 'name': 'maskbad'},
    {'children': [], 'name': 'med'},
    {'children': [], 'name': 'osiris'},
    {'children': [], 'name': 'shift_comb'},
    {'children': [], 'name': 'sky_sub'},
    {'children': [], 'name': 'spec_comb'}],
   'name': 'cirred'},
  {'children': [],

In [ ]:
with open('iraf_modules.json', 'w') as out:
    out.write(json.dumps(base_tree))

In [ ]:
with open('iraf_modules.csv', 'w') as out:
    out.write('id, value\n')
    out.write('clpackage,\n')
    for line in every_loaded_task:
        stuff = '.'.join([item for item in line.split(',')[:5] if item])
        print(stuff)
        out.write(stuff + ',\n')

### Just for a prettier view

In [ ]:
max_size = 0
for item in every_loaded_task:
    size = len(item.split('.')[1:])
    
    if size > max_size:
        max_size = size
    

all_rows = []
for item in every_loaded_task:
    split_path = item.split('.')[1:]
    new_row = [item for item in split_path[:-1]] + ['' for i in range(max_size - len(split_path[:-1]) -1)] + [split_path[-1]]
    all_rows.append(tuple(new_row))
    
    
names = tuple(['base'] + ['sub{}'.format(i) for i in range(max_size - 2)] + ['task'])
dtype = ['S10'] * max_size

t = Table(rows=all_rows, names=names, dtype=dtype)

In [ ]:
t.show_in_notebook()

### write to Excel

In [ ]:
workbook = xlsxwriter.Workbook('iraf_modules.xlsx')
worksheet = workbook.add_worksheet('Models')
for rownum in range(len(all_rows)):
    for colnum in range(len(all_rows[rownum])):
        worksheet.write(rownum, colnum, all_rows[rownum][colnum])
        

workbook.close()

# Finding IRAF on github

In [ ]:
import github3
import re
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
username = ''
password = ''
gh = github3.login(username, password)

In [ ]:
%%time
all_matches = []
for item in gh.search_code("iraf in:file language:python", text_match=True):
    for row in item.text_matches:
        #print(repr(row['fragment']))
        hits = re.findall("iraf\.(?P<name>[A-Za-z\t .]+)\(", row['fragment'])
        for call in hits:
            all_matches.append(call.split('.')[-1])


In [ ]:
freqs = Counter(all_matches).most_common()

In [ ]:
for name, count in freqs:
    print(count, name)

In [ ]:
plt.hist([item[1] for item in freqs], bins=len(freqs))